In [176]:
# 라이브러리 로딩
import requests as req
from bs4 import BeautifulSoup as bs

In [177]:
# 멜론차트 페이지 정보 받아오기
url = 'https://www.melon.com/chart/'
req.get(url)

# 400번대 오류 : 클라이언트의 요청 문제
# 멜론에서 브라우저가 아닌 코드로 페이지를 접근함을 인지하여 접근 불가능 !!

<Response [406]>

### 멜론 페이지에 브라우저인척 속여서 접근하기
- 개발자 도구에서 'Network' 탭 클릭
    - 이미 정보를 주고받은 뒤 통신이 끊겼기 때문에 탭을 클릭하였다 하더라도 빈 페이지 상태
    
    - 새로고침을 해서 주고받는 데이터들을 다시 불러주어야 함
    - 데이터 중에서 Type이 document인 문서를 클릭 !
    
- 브라우저로 접근을 했다는 것을 알려주는 코드가 있다 -> User-Agent
    - Chrome, Safari 등 접근이 가능한 정보가 담겨있다
    
    - 코드를 통해 접근했기 때문에 접근 가능한 정보가 들어가지 않는 것
    - User-Agent 전체 코드를 복사해서 코드로 가져오기

In [178]:
# 가져온 코드가 어떤 타입인지 알 수 없다.
# 파이썬에서 가져온 코드이기 떄문에 알맞는 타입으로 변경 해줘여 함
# 콜론 발견 -> 키와 벨류룰 가지는 딕셔너리가 알맞겠다 !

h = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

### headers 작업
- 컴퓨터(코드)가 아닌 사람(브라우저)으로 속여주는 작업

- 요청할 때 headers 값을 같이 채워서 요청 해주어야 한다.
    - req.get(url, headers = 헤더정보)

In [179]:
url = 'https://www.melon.com/chart/'
h = {'User-Agent' : 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/111.0.0.0 Safari/537.36'}

res = req.get(url, headers = h)

In [180]:
# bs 객체화
soup = bs(res.text, 'lxml')

### 멜론차트 데이터 수집
- 노래 제목
- 가수 이름

In [181]:
# 1. 노래 제목 수집
# 내가 수집하고자 하는 태그에 구분자(id, class)가 없었다..
# 그 요소를 포함하고 있는 부모 태그를 타고타고 올라가서 가져와야 함 (한칸씩 !)

title = soup.select('div.ellipsis.rank01 > span > a')
len(title)

# 주 의 사 항
# 하나의 클래스 이름 안에 띄어쓰기가 있는 경우
# 클래스가 다수인 경우 -> 띄어쓰기(자손 부등호)를 온접(클래스 부등호)으로 변경

100

In [182]:
# 2. 가수 이름 수집

singer = soup.select('div.ellipsis.rank02 > span')
len(singer)
singer[0].text

'NewJeans'

### 개수가 정해져 있는 데이터를 수집할 때에는 반드시 개수가 맞는지 확인

In [183]:
# 수집한 정보를 리스트로 저장
title_list = []
singer_list = []

for i in range(len(title)) :
    title_list.append(title[i].text)
    singer_list.append(singer[i].text)

In [184]:
# 길이 확인 ! --> 데이터 프레임으로 만들기 위한 준비 !!
len(title_list), len(singer_list)

(100, 100)

In [185]:
# for문 다른 방법
# 2개 이상의 리스트에 접근할 때 아래와 같은 for문 구조는 불편하다.
title_list2 = []

for i in title :
    title_list2.append(i.text)

In [186]:
singer_list2 = []

for i in singer :
    singer_list2.append(i.text)

In [188]:
import pandas as pd

# 1. 수집한 여러 개의 리스트를 하나의 딕셔너리로 생성
# 2. 딕셔너리를 활용해서 데이터프리임 생성
# 노래제목 : 데이터, 가수 : 데이터
data = {'노래제목':title_list, '가수':singer_list}
melon = pd.DataFrame(data)

In [192]:
# 데이터프레임명.to_csv() : 데이터프레임을 파일로 저장하기
# to_csv(저장하고 싶은 파일명, 확장자)
# 한글 저장할 때 인코딩 해주기 !

# 인코딩 방식 : euc-kr / utf-8 / cp949
# sep(seperate 구분자) : 하나의 기호만 사용 가능
melon.to_csv('멜론차트 TOP100.txt', encoding='euc-kr', sep='\t')

In [193]:
# 경로 설정하기
melon.to_csv('C:/Users/smhrd/Desktop/멜론차트.txt', encoding='euc-kr', sep='\t')

### csv는 txt(메모장) 파일과 excel(엑셀) 파일 둘 다 지원한다
- 주 의 ! 내가 저장하고 싶은 파일의 확장자를 정확히 적어주자 !
> csv의 장점 : excel보다 용량이 작고, 지원되는 폭이 넓다.  
> csv의 단점 : excel에 비해 다양한 작업은 불가능하다.